**'PT_Progression_Analysis_Unseen' notebook:**

In this notebook we generate the progression outcome for Part-time students who don't yet have a progression flag. The notebook uses the ABCS mapping stored in silverlayer to fetch the necessary mappings. Also it refers to the events table to fetch the students who don't have a progression flag yet, and for those students we retrieve features from the characteristics table. The progression outcomes are then generated by mapping the feature values with ABCS mapping, and finally the output table gets saved in the gold layer for being utilized in the dashboard.

In [1]:
# Importing necessary libraries
import logging
from enum import Enum
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.utils import AnalysisException
from pyspark.sql import Row
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import MapType, StringType
import json
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


# Set up logging configuration

class LogLevel(Enum):
    INFO = "INFO"
    ERROR = "ERROR"
    WARNING = "WARNING"

log_array = []

# Notebook name for log entries
notebook_name = "PT_Progression_Analysis_Unseen"

# Logging function with status options: In Progress, Successful, Failed
def logging(type_, message, status):
    msg = {
        "timestamp": str(pd.Timestamp.now()),  # Timestamp column
        "log_type": type_,                     # Log type column
        "status": status,                      # Status column
        "description": message,                # Description column
        "notebook": notebook_name              # Notebook name column
    }
    log_array.append(msg)
    print(msg)


# Function for storing log table
def createLogfile():
    try:
        # Initialize Spark session
        spark = SparkSession.builder.appName("ProgressionPTUnseenNotebook").getOrCreate()

        # Convert the log_array to a Spark DataFrame
        log_rows = [Row(**item) for item in log_array]  # Unpack the dictionary for Row entries
        new_log_df = spark.createDataFrame(log_rows)

        # Define the existing log table name
        with open("/lakehouse/default/Files/progression_config_template/progression_config.json") as config_file:
            config = json.load(config_file)
        
        # Define the existing log table name
        existing_log_table_name =config["progression_log"]
    
        # Write the new logs to the table (overwrite)
        new_log_df.write.mode("append").saveAsTable(existing_log_table_name)

        logging(LogLevel.INFO.value,f"Logging information saved in table '{existing_log_table_name}'",status="Successful")

        print(f"Logging information saved in table '{existing_log_table_name}'.")
    except Exception as e:
        logging(LogLevel.ERROR.value,f"Error appending log table: {e}",status="Failed")

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 3, Finished, Available, Finished)

In [2]:
try:
    # Loading config file
    with open("/lakehouse/default/Files/progression_config_template/progression_config.json") as progression_config:
                config = json.load(progression_config)

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error loading progression config file: {e}",status="Failed")
    createLogfile()
    raise e

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 4, Finished, Available, Finished)

**<mark>External ABCS Progression Mapping</mark>**

In [3]:
try:
    external_data_path = config["abcs_external"]
    query= "SELECT * FROM " + external_data_path
    df_p = spark.sql(query)
    df_pt=df_p.toPandas()
    df_pt
    logging(LogLevel.INFO.value,f"External Progression Mapping Loaded.",status="Successful")
except Exception as e:
    logging(LogLevel.ERROR.value,f"Error loading external progression mapping file: {e}",status="Failed")
    createLogfile()
    raise e     

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 5, Finished, Available, Finished)

{'timestamp': '2024-12-04 15:17:40.147174', 'log_type': 'INFO', 'status': 'Successful', 'description': 'External Progression Mapping Loaded.', 'notebook': 'PT_Progression_Analysis_Unseen'}


**Mapping 'Any' categories as blank ''. Also mapping 'Local' to 'FT' and 'Distance' to 'PT'**

In [4]:
df_pt1 = df_pt.replace({
    '-- Any quintile --':'', 
    '-- Any disability --':'', 
    '-- Any ethnicity --':'', 
    '-- Any quintile --':'', 
    '-- Any learner --': '',
    '-- Any sex --': '',
    'Local':'FT',
    'Distance':'PT',
    '-- Any age group --':''  
})

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 6, Finished, Available, Finished)

**Filtering the mappings for PT population only**

In [5]:
df_pt1= df_pt1[df_pt1['Local_or_distance_learner']=='PT']
df_pt1

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 7, Finished, Available, Finished)

,Adult_HE_quintile,Age_group,Disability,Ethnicity,IDACI,IMD,Local_or_distance_learner,Sex,Predicted_outcome_rate,Derived_progression_quintile,createdAt
1003,,21-25,Cognitive or learning difficulties,Mixed - white and black African,,,PT,Female,0.750000,1,2024-11-25 08:59:18.184056
1004,Quintile 1,21-25,Cognitive or learning difficulties,,,,PT,Female,0.720000,1,2024-11-25 08:59:18.184056
1005,,21-25,Cognitive or learning difficulties,,Quintile 1,,PT,Female,0.740000,1,2024-11-25 08:59:18.184056
1006,,21-25,Cognitive or learning difficulties,,Quintile 1,,PT,Female,0.740000,1,2024-11-25 08:59:18.184056
1007,,21-25,Cognitive or learning difficulties,,,Quintile 1,PT,Female,0.730000,1,2024-11-25 08:59:18.184056
...,...,...,...,...,...,...,...,...,...,...,...
770847,,,No disability reported,Asian or Asian British - Indian,,,PT,Male,0.820000,3,2024-11-25 08:59:18.184056
770848,,,No disability reported,Asian or Asian British - Indian,,,PT,Male,0.820000,3,2024-11-25 08:59:18.184056
770849,,,No disability reported,Asian or Asian British - Indian,,,PT,Male,0.820000,3,2024-11-25 08:59:18.184056
770850,,,No disability reported,Asian or Asian British - Indian,,,PT,Male,0.820000,3,2024-11-25 08:59:18.184056


In [6]:
df_pt1.nunique()

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 8, Finished, Available, Finished)

Adult_HE_quintile                6
Age_group                        9
Disability                       7
Ethnicity                       15
IDACI                            6
IMD                              6
Local_or_distance_learner        1
Sex                              3
Predicted_outcome_rate          78
Derived_progression_quintile     5
createdAt                        1
dtype: int64

**Creating 'Concatenated column' by adding different characteristics**

In [7]:
try:
    # Concatenate columns by converting all values to strings
    df_pt1['Concatenated_column'] = (
        df_pt1['Adult_HE_quintile'].astype(str) + 
        df_pt1['Age_group'].astype(str) + 
        df_pt1['Disability'].astype(str) + 
        df_pt1['Ethnicity'].astype(str) + 
        df_pt1['IDACI'].astype(str) + 
        df_pt1['IMD'].astype(str) + 
        df_pt1['Local_or_distance_learner'].astype(str) + 
        df_pt1['Sex'].astype(str)
    )
    logging(LogLevel.INFO.value,f"Concatenated column created for mapping.",status="Successful")

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error when creating concatinated column : {e}",status="Failed")
    createLogfile()
    raise e       

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 9, Finished, Available, Finished)

{'timestamp': '2024-12-04 15:17:48.357793', 'log_type': 'INFO', 'status': 'Successful', 'description': 'Concatenated column created for mapping.', 'notebook': 'PT_Progression_Analysis_Unseen'}


In [8]:
try:
    df_pt1=df_pt1[['Predicted_outcome_rate','Derived_progression_quintile','Concatenated_column']]
    df_pt1
except Exception as e:
    logging(LogLevel.ERROR.value,f"Error during slicing of mapping dataframe : {e}",status="Failed")
    createLogfile()
    raise e 

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 10, Finished, Available, Finished)

**<mark>Characteristics table import</mark>**

In [9]:
try:
    characteristics_data_path = config["characteristics_table"]
    query= "SELECT * FROM " + characteristics_data_path
    df_char = spark.sql(query)
    df_characteristics=df_char.toPandas()
    df_characteristics=df_characteristics.rename(columns={'stunumber':'StudentNumber'}, inplace=False)
    # Remove columns that end with 'value'
    df_characteristics = df_characteristics[[col for col in df_characteristics.columns if not col.endswith('value')]]
    df_characteristics=df_characteristics.drop(columns=['student_id','created_at','modified_at'])
    df_characteristics
    logging(LogLevel.INFO.value,f"Characteristics table successfully loaded.",status="Successful")

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error loading characteristics table : {e}",status="Failed")
    createLogfile()
    raise e    

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 11, Finished, Available, Finished)

{'timestamp': '2024-12-04 15:17:56.261651', 'log_type': 'INFO', 'status': 'Successful', 'description': 'Characteristics table successfully loaded.', 'notebook': 'PT_Progression_Analysis_Unseen'}


In [10]:
try:
       characteristics_required_features = df_characteristics[['StudentNumber', 'age_on_entry', 'ethnicity_16_groups', 'sex',
              'adult_he_2011_quintile_desc', 'imd_desc',
              'idaci_desc',  'disability_type']]

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error during slicing of characteristics dataframe : {e}",status="Failed")
    createLogfile()
    raise e 

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 12, Finished, Available, Finished)

**Assigning categories similar to ABCS mapping**

In [11]:
df_characteristics = df_characteristics.replace({
    '51 years and over on entry':'51+', 
    '41 to 50 years on entry':'41-50', 
    '26 to 30 years on entry':'26-30', 
    'Under 21 years on entry':'20', 
    '21 to 25 years on entry': '21-25', 
    '31 to 40 years on entry': '31-40', 
    'Unknown':'',

    'Refused_or_Unknown':'Refused or unknown',
    'White':'White',
    'Black_or_black_British_African':'Black or black British - African',
    'Other_ethnic_group':'Other ethnic group', 
    'Mixed_other':'Mixed - other',
    'Gypsy_or_Traveller':'',
    'Black_or_black_British_Caribbean':'Black or black British - Caribbean',
    'Asian_or_Asian_British_Bangladeshi':'Asian or Asian British - Bangladeshi',
    'Asian_or_Asian_British_Pakistani':'Asian or Asian British - Pakistani',
    'Mixed_White_and_black_African':'Mixed - white and black African',
    'Mixed_White_and_black_Caribbean':'Mixed - white and black Caribbean',
    'Asian_or_Asian_British_other':'Asian or Asian British - other', 
    'Asian_or_Asian_British_Chinese':'Asian or Asian British - Chinese',
    'Mixed_White_and_Asian':'Mixed - white and Asian', 
    'Asian_or_Asian_British_Indian':'Asian or Asian British - Indian',
    
    'Refused_unknown_or_not_collected':'',
    'Other_sex':'',

    'Unknown or invalid home postcode':'',
    'Quintile 1 England':'Quintile 1', 
    'Quintile 3 England':'Quintile 3', 
    'Quintile 2 England':'Quintile 2',
    'Quintile 4 England':'Quintile 4', 
    'Quintile 5 England':'Quintile 5',

    'No_disability_reported_or_unknown':'No disability reported', 
    'Mental_health_condition':'Mental health condition',
    'Cognitive_or_learning_difficulties':'Cognitive or learning difficulties',
    'Sensory_medical_or_physical_impairment':'Sensory, medical or physical impairments',
    'Multiple_or_other_impairments':'Multiple or other impairments',
    'Social_or_communication_impairment':'Social or communication impairment'
    
})


StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 13, Finished, Available, Finished)

In [12]:
df_characteristics.nunique()

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 14, Finished, Available, Finished)

StudentNumber                  96215
age_on_entry                       7
ethnicity_16_groups               15
sex                                3
ethnicity_6_groups                 6
gender_identity                    4
tundra_msoa_quintile_desc          6
adult_he_2011_quintile_desc        6
polar4_quintile_desc               6
imd_desc                           6
idaci_desc                         6
religion_or_belief                 9
sexual_orientation                 4
parental_education                 4
disability_type                    6
disability_status                  2
care_leaver                        2
entry_qualification_desc          55
dtype: int64

#### **Fetching Unseen Data**

In [13]:
# Fetching students and their latest course start date
df_latest_course_start_spark = spark.sql("Select Student_Code, max(Date) latest_Course_start_date from event_students where Event_Type = 'Course Start' group by Student_Code")

df_latest_course_start = df_latest_course_start_spark.toPandas()
df_latest_course_start

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 15, Finished, Available, Finished)

,Student_Code,latest_Course_start_date
0,STU103891,NaT
1,STU59124,NaT
2,STU59223,NaT
3,STU09979,NaT
4,STU32799,NaT
...,...,...
96239,STU219581,2024-11-25
96240,STU68665,NaT
96241,STU219499,NaT
96242,STU27020,NaT


In [14]:
# Fetching students and their course context
df_student_context_spark = spark.sql("Select Student_Code,Event_Context,Date latest_Course_start_date from event_students where Event_Type = 'Course Start'")
df_student_context = df_student_context_spark.toPandas()
df_student_context

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 16, Finished, Available, Finished)

,Student_Code,Event_Context,latest_Course_start_date
0,STU117862,"{""course"":""Unknown"",""mode_of_study"":""Part-time...",NaT
1,STU114691,"{""course"":""Unknown"",""mode_of_study"":""Part-time...",NaT
2,STU115858,"{""course"":""Unknown"",""mode_of_study"":""Part-time...",NaT
3,STU100440,"{""course"":""Unknown"",""mode_of_study"":""Part-time...",NaT
4,STU108643,"{""course"":""Unknown"",""mode_of_study"":""Part-time...",NaT
...,...,...,...
102733,STU56333,"{""course"":""Unknown"",""mode_of_study"":""Part-time...",NaT
102734,STU65156,"{""course"":""Unknown"",""mode_of_study"":""Part-time...",NaT
102735,STU68130,"{""course"":""Unknown"",""mode_of_study"":""Part-time...",NaT
102736,STU72645,"{""course"":""Unknown"",""mode_of_study"":""Part-time...",NaT


In [15]:
# Merging two tables to get student info as on latest course start date
df_student_info = pd.merge(df_latest_course_start, df_student_context, on=['Student_Code', 'latest_Course_start_date'], how='inner')
df_student_info

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 17, Finished, Available, Finished)

,Student_Code,latest_Course_start_date,Event_Context
0,STU103891,NaT,"{""course"":""Unknown"",""mode_of_study"":""Part-time..."
1,STU59124,NaT,"{""course"":""Unknown"",""mode_of_study"":""Part-time..."
2,STU59223,NaT,"{""course"":""Unknown"",""mode_of_study"":""Part-time..."
3,STU09979,NaT,"{""course"":""Unknown"",""mode_of_study"":""Part-time..."
4,STU32799,NaT,"{""course"":""Unknown"",""mode_of_study"":""Part-time..."
...,...,...,...
97964,STU219581,2024-11-25,"{""course"":""Master of Public Health"",""mode_of_s..."
97965,STU68665,NaT,"{""course"":""Unknown"",""mode_of_study"":""Part-time..."
97966,STU219499,NaT,"{""course"":""Unknown"",""mode_of_study"":""Part-time..."
97967,STU27020,NaT,"{""course"":""Unknown"",""mode_of_study"":""Part-time..."


In [16]:
# Converting to spark dataframe
df_student_details_spark = spark.createDataFrame(df_student_info)

schema = MapType(StringType(), StringType())
 
# Fetching Ipstartmode and Iplevel from event context, and dropping the event context after that
df_student_details_spark = df_student_details_spark.withColumn("Event_Context", from_json(col("Event_Context"), schema))
df_student_details_spark = df_student_details_spark.withColumn("IPSTARTMODE", col("Event_Context").getItem("mode_of_study"))
df_student_details_spark = df_student_details_spark.withColumn("IPLEVEL", col("Event_Context").getItem("level_of_study_desc"))

df_student_details_spark=df_student_details_spark.drop("Event_Context")

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 18, Finished, Available, Finished)

In [17]:
df_student_details = df_student_details_spark.toPandas()
df_student_details.nunique()

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 19, Finished, Available, Finished)

Student_Code                96244
latest_Course_start_date      153
IPSTARTMODE                     2
IPLEVEL                         4
dtype: int64

In [18]:
df_student_details

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 20, Finished, Available, Finished)

,Student_Code,latest_Course_start_date,IPSTARTMODE,IPLEVEL
0,STU103891,NaT,Part-time,First degree
1,STU59124,NaT,Part-time,First degree
2,STU59223,NaT,Part-time,First degree
3,STU09979,NaT,Part-time,Other undergraduate
4,STU32799,NaT,Part-time,Other undergraduate
...,...,...,...,...
97964,STU219581,2024-11-25,Full-time,Postgraduate taught masters’
97965,STU68665,NaT,Part-time,Postgraduate taught masters’
97966,STU219499,NaT,Part-time,First degree
97967,STU27020,NaT,Part-time,First degree


**Fetching those students who have already withdrawn from the course, and don't have a course start date after that, to discard those students from prediction**

In [19]:
df_student_withdrawn = spark.sql("Select Student_Code, Date as Withdrawal_Date from event_students where Event_Description = 'Student withdrawn from Course'")
df_student_withdrawn_pandas = df_student_withdrawn.toPandas()

print(df_student_withdrawn_pandas.nunique())

joined_withdrawn = pd.merge(df_student_details, df_student_withdrawn_pandas,on='Student_Code', how='inner')
print(joined_withdrawn.nunique())

joined_withdrawn1 = joined_withdrawn[(joined_withdrawn['Withdrawal_Date'] > joined_withdrawn['latest_Course_start_date'])]
print(joined_withdrawn1.nunique())

# Filter rows in 'df_student_details' where student_code is not present in joined_withdrawn1
df_student_details = df_student_details[~df_student_details['Student_Code'].isin(joined_withdrawn1['Student_Code'])]

df_student_details

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 21, Finished, Available, Finished)

Student_Code       39444
Withdrawal_Date    37052
dtype: int64
Student_Code                39444
latest_Course_start_date      149
IPSTARTMODE                     2
IPLEVEL                         4
Withdrawal_Date             37052
dtype: int64
Student_Code                37490
latest_Course_start_date      149
IPSTARTMODE                     2
IPLEVEL                         4
Withdrawal_Date             33687
dtype: int64


,Student_Code,latest_Course_start_date,IPSTARTMODE,IPLEVEL
0,STU103891,NaT,Part-time,First degree
1,STU59124,NaT,Part-time,First degree
2,STU59223,NaT,Part-time,First degree
3,STU09979,NaT,Part-time,Other undergraduate
4,STU32799,NaT,Part-time,Other undergraduate
...,...,...,...,...
97964,STU219581,2024-11-25,Full-time,Postgraduate taught masters’
97965,STU68665,NaT,Part-time,Postgraduate taught masters’
97966,STU219499,NaT,Part-time,First degree
97967,STU27020,NaT,Part-time,First degree


**Fetching those students who fall under non starter category,and don't have a course start date after that, to discard those students from prediction**

In [20]:
df_non_starter = spark.sql("Select Student_Code, Date as Nonstarter_Date from event_students WHERE Event_Description='Non Starter: Withdrawn'")

df_non_starter_pandas = df_non_starter.toPandas()
print(df_non_starter_pandas.nunique())

joined_non_starter = pd.merge(df_student_details, df_non_starter_pandas,on='Student_Code', how='inner')
print(joined_non_starter.nunique())

joined_non_starter1 = joined_non_starter[(joined_non_starter['Nonstarter_Date'] > joined_non_starter['latest_Course_start_date'])]
print(joined_non_starter1.nunique())

# Filter rows in 'df_student_details' where student_code is not present in 'joined_non_starter1'
df_student_details = df_student_details[~df_student_details['Student_Code'].isin(joined_non_starter1['Student_Code'])]

df_student_details

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 22, Finished, Available, Finished)

Student_Code       6297
Nonstarter_Date    6634
dtype: int64
Student_Code                1516
latest_Course_start_date      53
IPSTARTMODE                    2
IPLEVEL                        3
Nonstarter_Date             1571
dtype: int64
Student_Code                0
latest_Course_start_date    0
IPSTARTMODE                 0
IPLEVEL                     0
Nonstarter_Date             0
dtype: int64


,Student_Code,latest_Course_start_date,IPSTARTMODE,IPLEVEL
0,STU103891,NaT,Part-time,First degree
1,STU59124,NaT,Part-time,First degree
2,STU59223,NaT,Part-time,First degree
3,STU09979,NaT,Part-time,Other undergraduate
4,STU32799,NaT,Part-time,Other undergraduate
...,...,...,...,...
97964,STU219581,2024-11-25,Full-time,Postgraduate taught masters’
97965,STU68665,NaT,Part-time,Postgraduate taught masters’
97966,STU219499,NaT,Part-time,First degree
97967,STU27020,NaT,Part-time,First degree


In [21]:
joined_non_starter

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 23, Finished, Available, Finished)

,Student_Code,latest_Course_start_date,IPSTARTMODE,IPLEVEL,Nonstarter_Date
0,STU146394,2022-11-28,Full-time,First degree,2022-09-27 13:02:07
1,STU129368,2023-02-27,Full-time,Postgraduate taught masters’,2022-06-09 11:54:00
2,STU156189,2023-09-04,Full-time,First degree,2023-03-21 12:15:54
3,STU156189,2023-09-04,Full-time,First degree,2022-12-13 08:48:53
4,STU136924,2023-05-15,Full-time,Other undergraduate,2023-02-24 14:36:18
...,...,...,...,...,...
1711,STU189196,2023-11-27,Full-time,Other undergraduate,2023-09-29 13:52:56
1712,STU177953,2023-05-15,Full-time,Other undergraduate,2023-05-12 16:13:28
1713,STU256183,2024-11-25,Full-time,Other undergraduate,2024-11-22 14:36:07
1714,STU208252,2024-05-13,Full-time,First degree,2024-03-15 14:59:39


**Doing some additional mappings to match with Arden data**

In [22]:
df_student_details['IPSTARTMODE'] = df_student_details['IPSTARTMODE'].replace({'Full-time': 'FT', 'Part-time': 'PT'})
df_student_details['IPLEVEL'] = df_student_details['IPLEVEL'].replace({'First degree': 'DEG', 'Postgraduate taught masters’': 'PGTM', 'Other undergraduate':'OUG', 'Course aim does not apply':'NA', 'Other postgraduate taught':'OPGT', 'Degrees including a postgraduate component':'PUGD', 'Other qualifications with a postgraduate component':'PUGO'})
df_student_details

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 24, Finished, Available, Finished)

,Student_Code,latest_Course_start_date,IPSTARTMODE,IPLEVEL
0,STU103891,NaT,PT,DEG
1,STU59124,NaT,PT,DEG
2,STU59223,NaT,PT,DEG
3,STU09979,NaT,PT,OUG
4,STU32799,NaT,PT,OUG
...,...,...,...,...
97964,STU219581,2024-11-25,FT,PGTM
97965,STU68665,NaT,PT,PGTM
97966,STU219499,NaT,PT,DEG
97967,STU27020,NaT,PT,DEG


In [23]:
df_student_details = df_student_details.drop_duplicates()
df_student_details

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 25, Finished, Available, Finished)

,Student_Code,latest_Course_start_date,IPSTARTMODE,IPLEVEL
0,STU103891,NaT,PT,DEG
1,STU59124,NaT,PT,DEG
2,STU59223,NaT,PT,DEG
3,STU09979,NaT,PT,OUG
4,STU32799,NaT,PT,OUG
...,...,...,...,...
97964,STU219581,2024-11-25,FT,PGTM
97965,STU68665,NaT,PT,PGTM
97966,STU219499,NaT,PT,DEG
97967,STU27020,NaT,PT,DEG


**Fetching students with progression outcome, with Ipstartmode and Iplevel**

In [24]:
df_progression_students_spark = spark.sql("Select Student_Code,Event_Context from event_students where Event_Type = 'Progression Outcome'")

df_progression_students_spark = df_progression_students_spark.withColumn("Event_Context", from_json(col("Event_Context"), schema))
df_progression_students_spark  = df_progression_students_spark.withColumn("IPSTARTMODE", col("Event_Context").getItem("Ipstartmode"))
df_progression_students_spark  = df_progression_students_spark.withColumn("IPLEVEL", col("Event_Context").getItem("Iplevel"))

df_progression_students_spark = df_progression_students_spark.drop("Event_Context")

df_progression_students = df_progression_students_spark.toPandas()

df_progression_students = df_progression_students.drop_duplicates()
df_progression_students

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 26, Finished, Available, Finished)

,Student_Code,IPSTARTMODE,IPLEVEL
0,STU58129,PT,OPGT
1,STU82237,PT,OPGT
2,STU58363,PT,OPGT
3,STU97549,PT,OPGT
4,STU51057,PT,PUGO
...,...,...,...
1572,STU60349,FT,DEG
1574,STU53906,FT,DEG
1575,STU54765,FT,DEG
1576,STU53641,FT,DEG


**Removing the students with progression flag, and keeping those students only who don't have progression flag**

In [25]:
# Perform a left merge and mark rows that don't match
merged_seen_unseen_df = pd.merge(df_student_details, df_progression_students, on=['Student_Code', 'IPSTARTMODE', 'IPLEVEL'], how='left', indicator=True)

# Filter rows that are only in 'df_student_details
progression_unseen_students = merged_seen_unseen_df[merged_seen_unseen_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# Display result
progression_unseen_students = progression_unseen_students.drop('latest_Course_start_date', axis=1)
progression_unseen_students

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 27, Finished, Available, Finished)

,Student_Code,IPSTARTMODE,IPLEVEL
0,STU103891,PT,DEG
1,STU59124,PT,DEG
2,STU59223,PT,DEG
3,STU09979,PT,OUG
4,STU32799,PT,OUG
...,...,...,...
58750,STU219581,FT,PGTM
58751,STU68665,PT,PGTM
58752,STU219499,PT,DEG
58753,STU27020,PT,DEG


In [26]:
df_unseen= progression_unseen_students[progression_unseen_students['IPSTARTMODE']=='PT']
df_unseen.columns = ['StudentNumber', 'IPSTARTMODE', 'IPLEVEL']

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 28, Finished, Available, Finished)

In [27]:
df_unseen.nunique()

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 29, Finished, Available, Finished)

StudentNumber    24275
IPSTARTMODE          1
IPLEVEL              4
dtype: int64

**<mark>Unseen data + characteristics concatination</mark>**

In [28]:
try:
    concat_unseen_char = df_characteristics.merge(df_unseen,on='StudentNumber', how='inner')
    concat_unseen_char = concat_unseen_char.drop_duplicates()
except Exception as e:
    logging(LogLevel.ERROR.value,f"Error during merge of characteristics and unseen dataframe : {e}",status="Failed")
    createLogfile()
    raise e 

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 30, Finished, Available, Finished)

In [29]:
concat_unseen_char

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 31, Finished, Available, Finished)

,StudentNumber,age_on_entry,ethnicity_16_groups,sex,ethnicity_6_groups,gender_identity,tundra_msoa_quintile_desc,adult_he_2011_quintile_desc,polar4_quintile_desc,imd_desc,idaci_desc,religion_or_belief,sexual_orientation,parental_education,disability_type,disability_status,care_leaver,entry_qualification_desc,IPSTARTMODE,IPLEVEL
0,STU246049,21-25,Refused or unknown,Female,,Yes,,,,,,Hindu,Heterosexual,Yes,No disability reported,No_Disability_Reported,Care_experienced,Undergraduate credits,PT,DEG
1,STU205157,21-25,Refused or unknown,Female,,Not available,,,,,,,Refused_Unknown_or_not_collected,Not_known,No disability reported,No_Disability_Reported,"Refused, unknown or not collected",Non-UK first degree,PT,PGTM
2,STU249886,41-50,Refused or unknown,Male,,Yes,,,,,,,Refused_Unknown_or_not_collected,No,No disability reported,No_Disability_Reported,Care_experienced,Non-UK first degree,PT,PGTM
3,STU252403,51+,Refused or unknown,Female,,Yes,Quintile 4,Quintile 4,Quintile 4,Quintile 3,Quintile 4,Christian,Heterosexual,Yes,No disability reported,No_Disability_Reported,Care_experienced,UK first degree with honours,PT,PGTM
4,STU249622,51+,Refused or unknown,Female,,Yes,Quintile 5,Quintile 3,Quintile 4,Quintile 1,Quintile 2,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,UK first degree with honours,PT,PGTM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24241,STU146278,31-40,White,Female,White,Yes,Quintile 2,Quintile 3,Quintile 2,Quintile 2,Quintile 2,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,UK first degree with honours,PT,PGTM
24242,STU71837,31-40,White,Female,White,Yes,Quintile 1,Quintile 1,Quintile 2,Quintile 2,Quintile 2,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,UK first degree with honours,PT,DEG
24243,STU73281,31-40,White,Female,White,Yes,Quintile 3,Quintile 5,Quintile 4,Quintile 2,Quintile 2,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,UK first degree with honours,PT,PGTM
24244,STU99329,31-40,White,Female,White,Yes,Quintile 4,Quintile 3,Quintile 4,Quintile 2,Quintile 2,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,Higher National Diploma (HND),PT,DEG


In [30]:
concat_unseen_char.columns

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 32, Finished, Available, Finished)

Index(['StudentNumber', 'age_on_entry', 'ethnicity_16_groups', 'sex',
       'ethnicity_6_groups', 'gender_identity', 'tundra_msoa_quintile_desc',
       'adult_he_2011_quintile_desc', 'polar4_quintile_desc', 'imd_desc',
       'idaci_desc', 'religion_or_belief', 'sexual_orientation',
       'parental_education', 'disability_type', 'disability_status',
       'care_leaver', 'entry_qualification_desc', 'IPSTARTMODE', 'IPLEVEL'],
      dtype='object')

In [31]:
try:
       concat_unseen_char = concat_unseen_char[['StudentNumber', 'age_on_entry', 'ethnicity_16_groups', 'sex',
              'adult_he_2011_quintile_desc', 'imd_desc',
              'idaci_desc',  'disability_type', 'IPLEVEL','IPSTARTMODE']]

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error during slicing of concatinated characteristics dataframe : {e}",status="Failed")
    createLogfile()
    raise e 

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 33, Finished, Available, Finished)

In [32]:
concat_unseen_char.nunique()

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 34, Finished, Available, Finished)

StudentNumber                  24246
age_on_entry                       7
ethnicity_16_groups               15
sex                                3
adult_he_2011_quintile_desc        6
imd_desc                           6
idaci_desc                         6
disability_type                    6
IPLEVEL                            4
IPSTARTMODE                        1
dtype: int64

**Creating concatenated column by joining characteristics in same order as above to map with external progression data**

In [33]:
try:
    # Concatenate columns by converting all values to strings
    concat_unseen_char['Concatenated_column'] = (
        concat_unseen_char['adult_he_2011_quintile_desc'].astype(str) + 
        concat_unseen_char['age_on_entry'].astype(str) + 
        concat_unseen_char['disability_type'].astype(str) + 
        concat_unseen_char['ethnicity_16_groups'].astype(str) + 
        concat_unseen_char['idaci_desc'].astype(str) + 
        concat_unseen_char['imd_desc'].astype(str) + 
        concat_unseen_char['IPSTARTMODE'].astype(str) + 
        concat_unseen_char['sex'].astype(str)
    )

    concat_unseen_char=concat_unseen_char[['StudentNumber','IPLEVEL','Concatenated_column']]
    concat_unseen_char

    logging(LogLevel.INFO.value,f"Concatenated column created for mapping.",status="Successful")

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error when creating concatenated column : {e}",status="Failed")
    createLogfile()
    raise e    

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 35, Finished, Available, Finished)

/tmp/ipykernel_6757/3729321982.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat_unseen_char['Concatenated_column'] = (


In [34]:
concat_unseen_char['Concatenated_column'].nunique()

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 36, Finished, Available, Finished)

5551

**<mark>Inner joining with df_ft1 and concat_unseen_char, to map the students and get predictions</mark>**

In [35]:
df_pt1 = df_pt1.drop_duplicates()

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 37, Finished, Available, Finished)

In [36]:
df_pt1.shape

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 38, Finished, Available, Finished)

(156563, 3)

In [37]:
try:
    joined_df = df_pt1.merge(concat_unseen_char,on='Concatenated_column', how='inner')
    joined_df = joined_df.drop_duplicates()
    logging(LogLevel.INFO.value,f"Successfully merged dataframes using concatenated column mapping.",status="Successful")

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error when merging on concatenated column : {e}",status="Failed")
    createLogfile()
    raise e  

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 39, Finished, Available, Finished)

{'timestamp': '2024-12-04 15:18:47.486473', 'log_type': 'INFO', 'status': 'Successful', 'description': 'Successfully merged dataframes using concatenated column mapping.', 'notebook': 'PT_Progression_Analysis_Unseen'}


In [38]:
joined_df

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 40, Finished, Available, Finished)

,Predicted_outcome_rate,Derived_progression_quintile,Concatenated_column,StudentNumber,IPLEVEL
0,0.720000,1,Quintile 121-25Cognitive or learning difficult...,STU148861,DEG
1,0.670000,1,Quintile 121-25Cognitive or learning difficult...,STU188012,DEG
2,0.700000,1,Quintile 121-25Cognitive or learning difficult...,STU249734,DEG
3,0.640000,1,Quintile 121-25Cognitive or learning difficult...,STU118964,DEG
4,0.720000,1,Quintile 121-25Cognitive or learning difficult...,STU201146,DEG
...,...,...,...,...,...
22287,0.820000,3,No disability reportedRefused or unknownPT,STU48923,None
22288,0.820000,3,No disability reportedRefused or unknownPT,STU46863,None
22289,0.820000,3,No disability reportedRefused or unknownPT,STU46892,None
22290,0.820000,3,No disability reportedRefused or unknownPT,STU46856,None


**creating predicted outcome column positive progression rate threshold = 0.7**

In [39]:
try:
    joined_df['Predicted_Outcome'] = joined_df.apply(lambda row: "Positive" if row['Predicted_outcome_rate'] >0.7 else "Negative", axis=1)
    joined_df = joined_df[['StudentNumber', 'IPLEVEL','Concatenated_column', 'Predicted_Outcome','Predicted_outcome_rate', 'Derived_progression_quintile']]
    logging(LogLevel.INFO.value,f"Successfully generated positive and negative outcomes.",status="Successful")

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error when generating predictions: {e}",status="Failed")
    createLogfile()
    raise e  

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 41, Finished, Available, Finished)

{'timestamp': '2024-12-04 15:18:49.007938', 'log_type': 'INFO', 'status': 'Successful', 'description': 'Successfully generated positive and negative outcomes.', 'notebook': 'PT_Progression_Analysis_Unseen'}


In [40]:
joined_df

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 42, Finished, Available, Finished)

,StudentNumber,IPLEVEL,Concatenated_column,Predicted_Outcome,Predicted_outcome_rate,Derived_progression_quintile
0,STU148861,DEG,Quintile 121-25Cognitive or learning difficult...,Positive,0.720000,1
1,STU188012,DEG,Quintile 121-25Cognitive or learning difficult...,Negative,0.670000,1
2,STU249734,DEG,Quintile 121-25Cognitive or learning difficult...,Positive,0.700000,1
3,STU118964,DEG,Quintile 121-25Cognitive or learning difficult...,Negative,0.640000,1
4,STU201146,DEG,Quintile 121-25Cognitive or learning difficult...,Positive,0.720000,1
...,...,...,...,...,...,...
22287,STU48923,None,No disability reportedRefused or unknownPT,Positive,0.820000,3
22288,STU46863,None,No disability reportedRefused or unknownPT,Positive,0.820000,3
22289,STU46892,None,No disability reportedRefused or unknownPT,Positive,0.820000,3
22290,STU46856,None,No disability reportedRefused or unknownPT,Positive,0.820000,3


In [41]:
try:
    final_output_df = pd.merge(characteristics_required_features, joined_df[['StudentNumber', 'IPLEVEL', 'Predicted_Outcome', 'Predicted_outcome_rate', 'Derived_progression_quintile']], on=['StudentNumber'], how='inner')
    final_output_df['IPSTARTMODE']='PT'
    final_output_df['time_stamp'] = pd.Timestamp.now()
    logging(LogLevel.INFO.value,f"Final output table creation successful.",status="Successful")

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error when creating final output table: {e}",status="Failed")
    createLogfile()
    raise e  

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 43, Finished, Available, Finished)

{'timestamp': '2024-12-04 15:18:50.637614', 'log_type': 'INFO', 'status': 'Successful', 'description': 'Final output table creation successful.', 'notebook': 'PT_Progression_Analysis_Unseen'}


In [42]:
final_output_df

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 44, Finished, Available, Finished)

,StudentNumber,age_on_entry,ethnicity_16_groups,sex,adult_he_2011_quintile_desc,imd_desc,idaci_desc,disability_type,IPLEVEL,Predicted_Outcome,Predicted_outcome_rate,Derived_progression_quintile,IPSTARTMODE,time_stamp
0,STU246049,21 to 25 years on entry,Refused_or_Unknown,Female,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,No_disability_reported_or_unknown,DEG,Positive,0.740000,1,PT,2024-12-04 15:18:50.637249
1,STU205157,21 to 25 years on entry,Refused_or_Unknown,Female,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,No_disability_reported_or_unknown,PGTM,Positive,0.740000,1,PT,2024-12-04 15:18:50.637249
2,STU249886,41 to 50 years on entry,Refused_or_Unknown,Male,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,No_disability_reported_or_unknown,PGTM,Positive,0.840000,4,PT,2024-12-04 15:18:50.637249
3,STU252403,51 years and over on entry,Refused_or_Unknown,Female,Quintile 4,Quintile 3 England,Quintile 4,No_disability_reported_or_unknown,PGTM,Positive,0.750000,1,PT,2024-12-04 15:18:50.637249
4,STU249622,51 years and over on entry,Refused_or_Unknown,Female,Quintile 3,Quintile 1 England,Quintile 2,No_disability_reported_or_unknown,PGTM,Positive,0.820000,3,PT,2024-12-04 15:18:50.637249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22287,STU146278,31 to 40 years on entry,White,Female,Quintile 3,Quintile 2 England,Quintile 2,No_disability_reported_or_unknown,PGTM,Positive,0.740000,1,PT,2024-12-04 15:18:50.637249
22288,STU71837,31 to 40 years on entry,White,Female,Quintile 1,Quintile 2 England,Quintile 2,No_disability_reported_or_unknown,DEG,Positive,0.720000,1,PT,2024-12-04 15:18:50.637249
22289,STU73281,31 to 40 years on entry,White,Female,Quintile 5,Quintile 2 England,Quintile 2,No_disability_reported_or_unknown,PGTM,Positive,0.770000,2,PT,2024-12-04 15:18:50.637249
22290,STU99329,31 to 40 years on entry,White,Female,Quintile 3,Quintile 2 England,Quintile 2,No_disability_reported_or_unknown,DEG,Positive,0.740000,1,PT,2024-12-04 15:18:50.637249


**Saving the output table to Gold layer**

In [43]:
try:
    # Saving Output as Spark Dataframe
    spark_df = spark.createDataFrame(final_output_df)

    # Create a temporary view of the Spark DataFrame
    spark_df.createOrReplaceTempView("final_output_df")

    combined_prediction_output_path = config["unseen_output"]["dl_unseen_output_path"]

    # Hardcoding for one table save
    #combined_prediction_output_path = "GoldData.progression_unseen_data_prediction_dl"

    # Write the new predictions to the table
    spark_df.write.format("delta").mode("append").saveAsTable(combined_prediction_output_path)

    # Create or replace the table with column mapping mode enabled using the temporary view
    # spark.sql(f"""
    #         CREATE OR REPLACE TABLE {combined_prediction_output_path}
    #         USING delta
    #         TBLPROPERTIES ('delta.columnMapping.mode' = 'name')
    #         AS SELECT * FROM final_output_df
    # """)
    logging(LogLevel.INFO.value,f"Successfully uploaded final output table : {combined_prediction_output_path}",status="Successful")

except Exception as e:
    logging(LogLevel.ERROR.value,f"Error uploading final output table : {e}",status="Failed")
    createLogfile()
    raise e  

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 45, Finished, Available, Finished)

{'timestamp': '2024-12-04 15:18:55.163720', 'log_type': 'INFO', 'status': 'Successful', 'description': 'Successfully uploaded final output table : GoldData.progression_unseen_data_prediction_dl', 'notebook': 'PT_Progression_Analysis_Unseen'}


In [44]:
logging(LogLevel.INFO.value,f"Progression PT Unseen Prediction Successful.",status="Successful")
createLogfile()

StatementMeta(, fab35074-abc3-4955-bae5-ffcd1799d7e8, 46, Finished, Available, Finished)

{'timestamp': '2024-12-04 15:18:56.097477', 'log_type': 'INFO', 'status': 'Successful', 'description': 'Progression PT Unseen Prediction Successful.', 'notebook': 'PT_Progression_Analysis_Unseen'}
{'timestamp': '2024-12-04 15:18:58.392333', 'log_type': 'INFO', 'status': 'Successful', 'description': "Logging information saved in table 'SilverData.progression_log_table'", 'notebook': 'PT_Progression_Analysis_Unseen'}
Logging information saved in table 'SilverData.progression_log_table'.
